## Vulnerabilities
This documentation assumes that you already understand at least one common programming language and are generally familiar with JSON RESTful services. JSON specifies the format of the data returned by the REST service. REST refers to a style of services that allow computers to communicate via HTTP over the Internet. Click here for a list of best practices and additional information on where to start. The NVD is also documenting popular workflows to assist developers working with the APIs.

## CVE API
The CVE API is used to easily retrieve information on a single CVE or a collection of CVE from the NVD. The NVD contains 305,871 CVE records. Because of this, its APIs enforce offset-based pagination to answer requests for large collections. Through a series of smaller “chunked” responses controlled by an offset startIndex and a page limit resultsPerPage users may page through all the CVE in the NVD.

In [2]:
# Importo la librerías necesarias
import requests
import json
import os
import pandas as pd
from datetime import datetime, timedelta
import time

---
## Búsqueda por fecha

La API solo permite descargar 2000 resultados por vez. 
Entonces, primero realizo un request para cuantificar los resultados que se requieren ir a buscar.

In [4]:
# Parámetros 
url="https://services.nvd.nist.gov/rest/json/cves/2.0/"
params_time = "pubStartDate=2025-07-15T00:00:00.000&pubEndDate=2025-08-28T00:00:00.000"
from_time = "2025-07-15T00:00:00.000"
to_time = "2025-08-20T00:00:00.000"
params_index = "startIndex=0"
params_result_per_page = "resultsPerPage=2000"

# Realizar la petición
response_fecha = requests.get(f'{url}?{params_result_per_page}&{params_index}&pubStartDate={from_time}&pubEndDate={to_time}')

# Json to list:
list_data = response_fecha.json()

# Guardo el total de resultados 
total_result = list_data['totalResults']
print(f'La cantidad de resultados {total_result} durante {from_time} - {to_time}' )


La cantidad de resultados 4281 durante 2025-07-15T00:00:00.000 - 2025-08-20T00:00:00.000


In [5]:
# Voy haciendo peticiones cada 2000 resultados
cve = []
for i in range(0,total_result,2000):
    # Realizar la petición
    response_fecha = requests.get(f'{url}?{params_index}{i}&{params_time}')
    print(f'Respuesta del servidor: {response_fecha} a partir del indice {i}' )

    # Json to list:
    list_data = response_fecha.json()
    cve = cve + list_data['vulnerabilities']
    


# Dataframe:
lista_cve = [ i["cve"] for i in cve ]
data = pd.DataFrame(lista_cve)

# Castear las fechas:
data["published"] = pd.to_datetime(data["published"])
data["lastModified"] = pd.to_datetime(data["lastModified"])

Respuesta del servidor: <Response [200]> a partir del indice 0
Respuesta del servidor: <Response [200]> a partir del indice 2000
Respuesta del servidor: <Response [200]> a partir del indice 4000


## Tabla

In [7]:
cve_cpe_list = []
severity_cvssv31 = []
severity_cvssv4 = []
severity_cvssv2 = []
cve_description_list = []

for i in cve:
    # CVE
    cve_id = i['cve']['id']
    
    # CPE
    try:
        cve_cpe = i["cve"]['configurations'][0]['nodes'][0]['cpeMatch'][0]['criteria']
        cve_cpe_list.append(cve_cpe)
    except Exception as e:
        cve_cpe = None
        cve_cpe_list.append(cve_cpe)
    
    # Descripción:
    try:
        cve_description = i["cve"]['descriptions'][1]['value']
        cve_description_list.append(cve_description)
    except Exception as e:
        cve_description = None
        cve_description_list.append(cve_cpe)
    
    # Guardo Severidad para las distintas versiones CVSS
    # versión 3.1
    try:
        severidad =  i["cve"]['metrics']['cvssMetricV31'][0]['cvssData']['baseSeverity']
        severity_cvssv31.append(severidad)
    except Exception as e:
        severity_cvssv31.append(None)
    
    # versión 4.0
    try:
        severidad =  i["cve"]['metrics']['cvssMetricV40'][0]['cvssData']['baseSeverity']
        severity_cvssv4.append(severidad)                      
    except Exception as e:
        severity_cvssv4.append(None)
    
    # versión 2.0
    try:
        severidad =  i["cve"]['metrics']['cvssMetricV2'][0]['cvssData']['baseScore']
        severity_cvssv2.append(severidad)
    except Exception as e:
        severity_cvssv2.append(None)



# Agrego estos datos como columna a la tabla
data['CPE'] = cve_cpe_list
data['CVSS_31'] = severity_cvssv31
data['CVSS_4'] = severity_cvssv4
data['CVSS_2'] = severity_cvssv2
data['Description'] = cve_description_list


## LOAD: Inventario de equipos.

In [8]:
# Leo el inventarios
tecnologias_monex = pd.read_excel("..//data//raw//tecnologias_monex.xlsx")
marcas = tecnologias_monex['Marca'].drop_duplicates().values

### Buscar por Marca o Vendor

In [9]:
# Itero entre la lista de marcas y busco en la tabla de los cve del NIST
vuln_marcas =[] 
for i in marcas:
    vuln_df = data[data['CPE'].str.contains(i, case = False, na= False)]
    vuln_marcas.append(vuln_df)
    

# Concateno la lista de los resultados encontrados
vuln_marcas = pd.concat(vuln_marcas, ignore_index=True)

C:\Users\mbustos_lightech\AppData\Local\Temp\ipykernel_24552\3315835943.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  vuln_df = data[data['CPE'].str.contains(i, case = False, na= False)]


### Filtro: HIGH & CRITICAL

In [30]:
vuln_high_critical = vuln_marcas[vuln_marcas['CVSS_31'].isin(['HIGH','CRITICAL']) |
                                vuln_marcas['CVSS_4'].isin(['HIGH','CRITICAL'])]
# vuln_high_critical = vuln_high_critical[['id', 'published', 'lastModified', 'CPE', 'CVSS_31', 'CVSS_4', 'Description']]

### FORMATO FINAL

In [35]:
# Extraer el fragmento del CPE (a partir del tercer ":" y del cuarto ":")
vuln_high_critical['Modelo'] = vuln_high_critical['CPE'].str.split(':').str[4]
vuln_high_critical['Marca'] = vuln_high_critical['CPE'].str.split(':').str[3]

# Renombrar columnas para que sean más claras
vuln_high_critical = vuln_high_critical.rename(columns={
    "published": "Fecha de creación",
    "Description": "Descripción"    
})

# Dataframe final
vuln_high_critical = vuln_high_critical[['id', "Fecha de creación", 'Marca', 'Modelo',
       'Descripción', 'CVSS_31', 'CVSS_4']]

vuln_high_critical.sort_values(by=['CVSS_31'], inplace=True)

### LOAD

In [37]:
# Guardo la info en un archivo CSV
vuln_high_critical.to_csv('..//data//stage//vuln_high_critical.csv', index=False, encoding='utf-8-sig')


In [34]:
# vuln_high_critical = vuln_high_critical.astype('str')
vuln_high_critical.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 0 to 307
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 146 non-null    object        
 1   Fecha de creación  146 non-null    datetime64[ns]
 2   Marca              146 non-null    object        
 3   Modelo             146 non-null    object        
 4   Descripción        146 non-null    object        
 5   CVSS_31            146 non-null    object        
 6   CVSS_4             6 non-null      object        
dtypes: datetime64[ns](1), object(6)
memory usage: 9.1+ KB
